In [31]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt

In [32]:
maxAcc = 0.0

# 80% of raw data is used for training the model- Training Data
TrainingPercent = 70

#10% of the raw data is used as validation data
ValidationPercent = 15

#10% of the raw data is used as Testing Data
TestPercent = 15

#Dividing the raw data into various number of clusters which will be equal to number of basis of the basis function
M = [5,7,10,12]

#Training the model on different values of least squared regularization.
L=[0.03,0.05,1.0,2.0]

#Design Matrix
PHI = []
#Number of times the Weight updation is going to be done in Stochastic Gradient Descent Solution
maxIter = 400

IsSynthetic = False

In [33]:
# The target vector consists of first field of the row of the dataset(0,1,2 target values). 
#Creating a vector consisting only of target values by using the preprocessed querylevelnormal file called querylevelnormal_rawtarget.csv
def GenerateRawTarget(filePath):
    t = []
    with open(filePath, 'rU') as f:
        reader = csv.reader(f)
        for row in reader:  
            t.append(int(row[0]))
    #print("Raw Target Generated..")
    return t

#Creating a dataset matrix that consists of all other features of the entire dataset(rows) located at specified file path(querylevelnorm_rawdata)
def GenerateRawData(filePath, IsSynthetic):    
    dataMatrix = [] 
    with open(filePath, 'rU') as fi:
        reader = csv.reader(fi)
        for row in reader:
            dataRow = []
            for column in row:
                dataRow.append(float(column))
            dataMatrix.append(dataRow)   
    #print("Data matrix(69623,46) is generated")
    if IsSynthetic == False :
    #Deleting 5 feature columns whose variance is zero so that we can compute inverse in future computation 
       dataMatrix = np.delete(dataMatrix, [5,6,7,8,9], axis=1)
    dataMatrix = np.transpose(dataMatrix)     
    #print("Data Matrix(41,55699) is generated after deleting 5 features of zero variance and transposing the matrix")
    return dataMatrix


#Generating training target from raw target by restricting the length of the training target to only 80% of whole raw target.
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    #From the beginning to the calculated training length
    t           = rawTraining[:TrainingLen]
   # print(str(TrainingPercent) + "% Training Target Generated..")
    return t


#Generating training data from whole dataset by restricting the length of the  training dataset to be only 80% of whole raw dataset.
def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
   # print(str(TrainingPercent) + "% Training Data Generated..")
    return d2


#Generating validation/testing target from raw target by restricting the length of the validation/testing target to be only 10% of whole raw target.
def GenerateTargetData(rawData,ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

#Generating validation/testing data from whole dataset by restricting the length of the validation/testing dataset to only 10% of whole dataset
def GenerateTargetVector(rawData,ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    return t

#Generating co-variance matrix. All elements other than diagonalmatrix are zero as the features are independent of each other.
def GenerateBigSigma(Data,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)     
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(len(DataT))):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(400,BigSigma)
    #print("BigSigma is generated")
    return BigSigma

# This generates the exponential part present in the radial basis function
def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

#This generates the radial basis function
def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

#This generates the design matrix consisting of datasamples and basis functions. 
def GetPhiMatrix(Data, MuMatrix, BigSigma):
    DataT = np.transpose(Data)
    #TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(len(DataT)),len(MuMatrix)))
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(len(DataT))):
           #Each element of design matrix is a scalar value.
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

#Solving the closed form equation using Moore-Penrose pseudo inverse.
def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    #print ("Training Weights Generated..")
    return W

#Generating the output values of validation and testing data
def GetOutputValues(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    return Y

#Generating the root-mean squared error value and accuracy.
def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    return (str(accuracy)+ ',' + str(math.sqrt(sum/len(VAL_TEST_OUT))))

## Fetch and Prepare Dataset

In [34]:
RawTarget = GenerateRawTarget('Querylevelnorm_RawTarget.csv')
RawData   = GenerateRawData('Querylevelnorm_RawData.csv',IsSynthetic)
print(len(RawTarget))
print(RawData.shape)

c:\users\ravali pinnaka\desktop\python\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 'U' mode is deprecated
  """
c:\users\ravali pinnaka\desktop\python\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 'U' mode is deprecated
  from ipykernel import kernelapp as app


69623
(41, 69623)


## Prepare Training Data

In [35]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingDataMatrix   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
print(TrainingTarget.shape)
print(TrainingDataMatrix.shape)

(48737,)
(41, 48737)


## Prepare Validation Data

In [36]:
ValidationTarget = np.array(GenerateTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValidationDataMatrix    = GenerateTargetData(RawData,ValidationPercent, (len(TrainingTarget)))
print(ValidationTarget.shape)
print(ValidationDataMatrix.shape)

(10443,)
(41, 10443)


## Prepare Test Data

In [37]:
TestingTarget = np.array(GenerateTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValidationTarget))))
TestingDataMatrix = GenerateTargetData(RawData,TestPercent, (len(TrainingTarget)+len(ValidationTarget)))
print(TestingTarget.shape)
print(TestingDataMatrix.shape)

(10442,)
(41, 10442)


## Closed Form Solution [Finding Weights using Moore- Penrose pseudo- Inverse Matrix]

In [38]:
#using beautiful table to generate tabular values
from beautifultable import BeautifulTable 
ErmsTable = BeautifulTable()
AccuracyTable = BeautifulTable()
print ('UBITname      = Saranya')
print ('Person Number = 50248926')
print ('----------------------------------------------------')
print ("------------------LeToR Data------------------------")
print ('----------------------------------------------------')
print ("-------Closed Form with Radial Basis Function-------")
print ('----------------------------------------------------')

AccuracyTable.column_headers = ["Clusters", "Lambda", "AccuracyTraining","AccuarcyValidation","AccuracyTesting","difference"]
ErmsTable.column_headers = ["Clusters", "Lambda", "ErmsTraining","ErmsValidation","ErmsTesting"]
ErmsTraining = []
ErmsValidation = []
AccuracyTesting = []
AccuracyTraining = []
AccuracyValidation = []
ErmsTesting = []
# #Code for generating Erms and accuracy values for different number of basis functions and regularized values
# for i in M:
#     for j in L:
#         kmeans = KMeans(n_clusters=i, random_state=0).fit(np.transpose(TrainingDataMatrix))
#         Mu = kmeans.cluster_centers_
#         BigSigma     = GenerateBigSigma(TrainingDataMatrix,IsSynthetic)
#         TRAINING_PHI = GetPhiMatrix(TrainingDataMatrix, Mu, BigSigma)
#         W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(j)) 
#         TESTING_PHI     = GetPhiMatrix(TestingDataMatrix, Mu, BigSigma) 
#         VALIDATION_PHI      = GetPhiMatrix(ValidationDataMatrix, Mu, BigSigma)
#         TRAINING_TEST_OUT  = GetOutputValues(TRAINING_PHI,W)
#         VALIDATION_TEST_OUT = GetOutputValues(VALIDATION_PHI,W)
#         TESTING_OUT     = GetOutputValues(TESTING_PHI,W)        
#         TrainingAccuracy   = str(GetErms(TRAINING_TEST_OUT,TrainingTarget))  
#         ValidationAccuracy =str(GetErms(VALIDATION_TEST_OUT,ValidationTarget))        
#         TestingAccuracy       = str(GetErms(TESTING_OUT,TestingTarget))
#         ErmsTraining.append(float(TrainingAccuracy.split(',')[1]))
#         ErmsValidation.append(float(ValidationAccuracy.split(',')[1]))
#         ErmsTesting.append(float(TestingAccuracy.split(',')[1]))
#         AccuracyTraining.append(float(TrainingAccuracy.split(',')[0]))
#         AccuracyValidation.append(float(ValidationAccuracy.split(',')[0]))
#         AccuracyTesting.append(float(TestingAccuracy.split(',')[0]))
#         ErmsTable.append_row([i, j,str(float(TrainingAccuracy.split(',')[1])),str(float(ValidationAccuracy.split(',')[1])),str(float(TestingAccuracy.split(',')[1]))])
#         AccuracyTable.append_row([i, j,str(float(TrainingAccuracy.split(',')[0])),str(float(ValidationAccuracy.split(',')[0])),str(float(TestingAccuracy.split(',')[0])),float(TrainingAccuracy.split(',')[0])-float(ValidationAccuracy.split(',')[0])])

kmeans = KMeans(n_clusters=7, random_state=0).fit(np.transpose(TrainingDataMatrix))
Mu = kmeans.cluster_centers_
BigSigma     = GenerateBigSigma(TrainingDataMatrix,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(TrainingDataMatrix, Mu, BigSigma)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(2.0)) 
TESTING_PHI     = GetPhiMatrix(TestingDataMatrix, Mu, BigSigma) 
VALIDATION_PHI      = GetPhiMatrix(ValidationDataMatrix, Mu, BigSigma)
TRAINING_TEST_OUT  = GetOutputValues(TRAINING_PHI,W)
VALIDATION_TEST_OUT = GetOutputValues(VALIDATION_PHI,W)
TESTING_OUT     = GetOutputValues(TESTING_PHI,W)        
TrainingAccuracy   = str(GetErms(TRAINING_TEST_OUT,TrainingTarget))  
ValidationAccuracy =str(GetErms(VALIDATION_TEST_OUT,ValidationTarget))        
TestingAccuracy       = str(GetErms(TESTING_OUT,TestingTarget))
ErmsTraining.append(float(TrainingAccuracy.split(',')[1]))
ErmsValidation.append(float(ValidationAccuracy.split(',')[1]))
ErmsTesting.append(float(TestingAccuracy.split(',')[1]))
AccuracyTraining.append(float(TrainingAccuracy.split(',')[0]))
AccuracyValidation.append(float(ValidationAccuracy.split(',')[0]))
AccuracyTesting.append(float(TestingAccuracy.split(',')[0]))
ErmsTable.append_row([7, 2.0,str(float(TrainingAccuracy.split(',')[1])),str(float(ValidationAccuracy.split(',')[1])),str(float(TestingAccuracy.split(',')[1]))])
AccuracyTable.append_row([7, 2.0,str(float(TrainingAccuracy.split(',')[0])),str(float(ValidationAccuracy.split(',')[0])),str(float(TestingAccuracy.split(',')[0])),float(TrainingAccuracy.split(',')[0])-float(ValidationAccuracy.split(',')[0])])
print('Accuracy Values') 
print(AccuracyTable)
print('Erms Values')
print(ErmsTable)


UBITname      = Saranya
Person Number = 50248926
----------------------------------------------------
------------------LeToR Data------------------------
----------------------------------------------------
-------Closed Form with Radial Basis Function-------
----------------------------------------------------
Accuracy Values
+----------+--------+--------------+----------------+-------------+------------+
| Clusters | Lambda | AccuracyTrai | AccuarcyValida | AccuracyTes | difference |
|          |        |     ning     |      tion      |    ting     |            |
+----------+--------+--------------+----------------+-------------+------------+
|    7     |  2.0   |    74.494    |     73.647     |    71.05    |   0.846    |
+----------+--------+--------------+----------------+-------------+------------+
Erms Values
+----------+--------+--------------+-----------------------------+-------------+
| Clusters | Lambda | ErmsTraining |       ErmsValidation        | ErmsTesting |
+---------

## Finding Erms on training, validation and test set 

In [39]:

# #print ("M = 10 \nLambda = 0.9")
# print ("E_rms Training   = " + str(TrainingAccuracy))
# print ("E_rms Validation = " + str(ValidationAccuracy))
# print ("E_rms Testing    = " + str(TestingAccuracy))

E_rms Training   = 74.49371114348442,0.5496936652644908
E_rms Validation = 73.64741932394905,0.5502920135666132
E_rms Testing    = 71.04960735491285,0.6043393901303056


## Gradient Descent solution for Linear Regression

In [40]:
print ('----------------------------------------------------')
print ('--------------Please Wait for 2 mins!----------------')
print ('------------Updating the weights for 400 iterations------------')

----------------------------------------------------
--------------Please Wait for 2 mins!----------------
----------------------------------------------------


In [41]:
from beautifultable import BeautifulTable 
ErmsTable = BeautifulTable()
AccuracyTable = BeautifulTable()
AccuracyTable.column_headers = ["Learning Rate", "Lambda", "AccuracyTraining","AccuarcyValidation","AccuracyTesting","difference"]
ErmsTable.column_headers = ["Learning Rate", "Lambda", "ErmsTraining","ErmsValidation","ErmsTesting"]

#scalar multiplication of weight matrix
W_Now        = np.dot(220, W)
#Regularizer values
La           = [1,2,3,4]
#Learning Rate
learningRate = [0.2,0.1,0.01,0.05]
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
L_Acc_Val   = []
L_Acc_TR    = []
L_Acc_Test  = []
W_Mat        = []

# #Early stopping as the weights may become stagnant
# for lam in La:
#     for lr in learningRate:
#         for i in range(0,400):
    
#             #print ('---------Iteration: ' + str(i) + '--------------')
#                 Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
#                 La_Delta_E_W  = np.dot(lam,W_Now)
#                 Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
#                 Delta_W       = -np.dot(lr,Delta_E)
#                 #Updating weights using error regularizer
#                 W_T_Next      = W_Now + Delta_W
#                 W_Now         = W_T_Next

#             #-----------------TrainingData Accuracy---------------------#

#                 TRAINING_TEST_OUT   = GetOutputValues(TRAINING_PHI,W_T_Next) 
#                 Erms_TR       = GetErms(TRAINING_TEST_OUT,TrainingTarget)
#                 L_Erms_TR.append(float(Erms_TR.split(',')[1]))
#                 L_Acc_TR.append(float(Erms_TR.split(',')[0]))

#                 VALIDATION_TEST_OUT  = GetOutputValues(VALIDATION_PHI,W_T_Next) 
#                 Erms_Val      = GetErms(VALIDATION_TEST_OUT,ValidationTarget)
#                 L_Erms_Val.append(float(Erms_Val.split(',')[1]))
#                 L_Acc_Val.append(float(Erms_Val.split(',')[0]))

#             #-----------------TestingData Accuracy---------------------#
#                 TESTING_OUT      = GetOutputValues(TESTING_PHI,W_T_Next) 
#                 Erms_Test = GetErms(TESTING_OUT,TestingTarget)
#                 L_Erms_Test.append(float(Erms_Test.split(',')[1]))
#                 L_Acc_Test.append(float(Erms_Test.split(',')[0]))
#                 ErmsTable.append_row([lr,lam,str(float(Erms_TR.split(',')[1])),str(float(Erms_Val.split(',')[1])),str(float( Erms_Test.split(',')[1]))])
#                 AccuracyTable.append_row([lr,lam,str(float(Erms_TR.split(',')[0])),str(float(Erms_Val.split(',')[0])),str(float( Erms_Test.split(',')[0])),float( Erms_TR.split(',')[0])-float(Erms_Val.split(',')[0])])

                
                
#Early stopping as the weights may become stagnant i.e there will be no updation of weights
for i in range(0,400):
    
            #print ('---------Iteration: ' + str(i) + '--------------')
                Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
                La_Delta_E_W  = np.dot(2,W_Now)
                Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
                Delta_W       = -np.dot(0.01,Delta_E)
                #Updating weights using error regularizer
                W_T_Next      = W_Now + Delta_W
                W_Now         = W_T_Next

            #-----------------TrainingData Accuracy---------------------#

                TRAINING_TEST_OUT   = GetOutputValues(TRAINING_PHI,W_T_Next) 
                Erms_TR       = GetErms(TRAINING_TEST_OUT,TrainingTarget)
                L_Erms_TR.append(float(Erms_TR.split(',')[1]))
                L_Acc_TR.append(float(Erms_TR.split(',')[0]))

                VALIDATION_TEST_OUT  = GetOutputValues(VALIDATION_PHI,W_T_Next) 
                Erms_Val      = GetErms(VALIDATION_TEST_OUT,ValidationTarget)
                L_Erms_Val.append(float(Erms_Val.split(',')[1]))
                L_Acc_Val.append(float(Erms_Val.split(',')[0]))

            #-----------------TestingData Accuracy---------------------#
                TESTING_OUT      = GetOutputValues(TESTING_PHI,W_T_Next) 
                Erms_Test = GetErms(TESTING_OUT,TestingTarget)
                L_Erms_Test.append(float(Erms_Test.split(',')[1]))
                L_Acc_Test.append(float(Erms_Test.split(',')[0]))
                ErmsTable.append_row([0.001,3,str(float(Erms_TR.split(',')[1])),str(float(Erms_Val.split(',')[1])),str(float( Erms_Test.split(',')[1]))])
                AccuracyTable.append_row([0.001,3,str(float(Erms_TR.split(',')[0])),str(float(Erms_Val.split(',')[0])),str(float( Erms_Test.split(',')[0])),float( Erms_TR.split(',')[0])-float(Erms_Val.split(',')[0])])

print('Accuracy Values') 
print(AccuracyTable)
print('Erms Values')
print(ErmsTable)

    
    
    

Accuracy Values
+-----------+--------+--------------+---------------+-------------+------------+
| Learning  | Lambda | AccuracyTrai | AccuarcyValid | AccuracyTes | difference |
|   Rate    |        |     ning     |     ation     |    ting     |            |
+-----------+--------+--------------+---------------+-------------+------------+
|   0.001   |   3    |    0.244     |     0.239     |    0.22     |   0.005    |
+-----------+--------+--------------+---------------+-------------+------------+
|   0.001   |   3    |    0.164     |     0.211     |    0.182    |   -0.047   |
+-----------+--------+--------------+---------------+-------------+------------+
|   0.001   |   3    |    0.228     |     0.239     |    0.182    |   -0.012   |
+-----------+--------+--------------+---------------+-------------+------------+
|   0.001   |   3    |    0.263     |     0.259     |    0.249    |   0.004    |
+-----------+--------+--------------+---------------+-------------+------------+
|   0.001   

In [46]:
print ('----------Gradient Descent Solution--------------------')
print ("M = 7 \nLambda  = 3\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
print ("Acc Testing    = " + str(np.around(max(L_Acc_Test),5)))

----------Gradient Descent Solution--------------------
M = 7 
Lambda  = 3
eta=0.01
E_rms Training   = 0.54978
E_rms Validation = 0.55029
E_rms Testing    = 0.6022
Acc Testing    = 71.37522


## code for graphs

In [43]:
# import numpy as np
# from matplotlib.pyplot import figure
# import scipy.interpolate as interp
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# #figure(num=None, figsize=(62, 62), dpi=20, facecolor='w', edgecolor='k')
# X = [5,5,5,5,7,7,7,7,10,10,10,10,12,12,12,12]
# Y = [0.03,0.03,0.03,0.03,0.05,0.05,0.05,0.05,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0]
# Z = AccuracyValidation

# plotx,ploty, = np.meshgrid(np.linspace(np.min(X),np.max(X),20),\
#                            np.linspace(np.min(Y),np.max(Y),20))
# plotz = interp.griddata((X,Y),Z,(plotx,ploty),method='linear')
# fig = plt.figure()
# fig2 = matplotlib.pyplot.gcf()
# fig2.set_size_inches(17.5, 6.5)
# ax = fig.add_subplot(1,2,2, projection='3d')
# ax.plot_surface(plotx,ploty,plotz,cstride=3,rstride=3,cmap='viridis')
# ax.set_xlabel('No of Basis functions')
# ax.set_ylabel('Regularization rate')
# ax.set_zlabel('Validation Accuracy ')
# Z1 = AccuracyTesting
# plotz1 = interp.griddata((X,Y),Z1,(plotx,ploty),method='linear')
# ax = fig.add_subplot(1,2,1, projection='3d')
# ax.plot_surface(plotx,ploty,plotz1,cstride=3,rstride=3,cmap='viridis')
# ax.set_xlabel('No of Basis functions')
# ax.set_ylabel('Regularization rate')
# ax.set_zlabel('Testing Accuracy ') 

# plt.draw()


In [44]:
# import numpy as np
# from matplotlib.pyplot import figure
# import scipy.interpolate as interp
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
# #figure(num=None, figsize=(62, 62), dpi=20, facecolor='w', edgecolor='k')
# X = [5,5,5,5,7,7,7,7,10,10,10,10,12,12,12,12]
# Y = [0.03,0.03,0.03,0.03,0.05,0.05,0.05,0.05,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0]
# Z = ErmsValidation

# plotx,ploty, = np.meshgrid(np.linspace(np.min(X),np.max(X),20),\
#                            np.linspace(np.min(Y),np.max(Y),20))
# plotz = interp.griddata((X,Y),Z,(plotx,ploty),method='linear')
# fig = plt.figure()
# fig2 = matplotlib.pyplot.gcf()
# fig2.set_size_inches(17.5, 6.5)
# ax = fig.add_subplot(1,2,2, projection='3d')
# ax.plot_surface(plotx,ploty,plotz,cstride=3,rstride=3,cmap='viridis')
# ax.set_xlabel('No of Basis functions')
# ax.set_ylabel('Regularization rate')
# ax.set_zlabel('Validation Erms ')
# Z1 = ErmsTesting
# plotz1 = interp.griddata((X,Y),Z1,(plotx,ploty),method='linear')
# ax = fig.add_subplot(1,2,1, projection='3d')
# ax.plot_surface(plotx,ploty,plotz1,cstride=3,rstride=3,cmap='viridis')
# ax.set_xlabel('No of Basis functions')
# ax.set_ylabel('Regularization rate')
# ax.set_zlabel('Testing Erms ') 

# plt.draw()

In [45]:

# plt.subplot(3,1,1)
# plt.plot (AccuracyTraining,[0.03,0.03,0.03,0.03,0.05,0.05,0.05,0.05,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0],'r')
# plt.xlabel('Training_Accuracy')
# plt.ylabel('regularizer_lambda1')
# plt.title('Training_Accuracy vs regularizer_lambda1')
# plt.show()

# plt.subplot(3,1,2)
# plt.plot (AccuracyValidation,[0.03,0.03,0.03,0.03,0.05,0.05,0.05,0.05,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0],'b')
# plt.xlabel('Validation_Accuracy')
# plt.ylabel('regularizer_lambda1')
# plt.title('Validation_Accuracy vs regularizer_lambda1)')
# plt.show()

# plt.subplot(3,1,3)
# plt.plot (AccuracyTesting,[0.03,0.03,0.03,0.03,0.05,0.05,0.05,0.05,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0],'b')
# plt.xlabel('Testing_Accuracy')
# plt.ylabel('regularizer_lambda1)')
# plt.title('Testing_Accuracy vs regularizer_lambda1')
# plt.show()